Problems 6-8, 22, 23

In [125]:
%matplotlib inline

import numpy as np
import scipy.stats as st
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from statsmodels import stats

## 6.6

### a. 


$\alpha=0.01$

$H_0: \beta_1 = \beta_2 = 0$

$H_a:$ not all $\beta$ equal zero

$F^* = \frac{MSR}{MSE}$

If $F^* \le F(1-\alpha; p-1; n-p)$, conclude $H_0$

If $F^* > F(1-\alpha; p-1; n-p)$, conclude $H_a$

In this case, $p=3$

In [126]:
# Load the brand preference data
df_bp = pd.read_table('/Users/kevin/Dropbox/School/STA-580/ch6hw/CH06PR05.txt',
                     sep='\s*', index_col=False, engine='python',
                     names=['brand_liking', 'moisture', 'sweetness'])
df_bp.head()

,brand_liking,moisture,sweetness
0,64,4,2
1,73,4,4
2,61,4,2
3,76,4,4
4,72,6,2


We'll use the `statsmodels` ordinary least squares regression to get $MSE$ and $MSR$. 

In [127]:
# Fit an OLS model with intercept on moisture and sweetness
X_bp = df_bp[['moisture', 'sweetness']]
X_bp = sm.add_constant(X_bp)
y_bp = df_bp['brand_liking']
lr_bp = sm.OLS(y_bp, X_bp).fit()
lr_bp.mse_resid

7.2538461538461547

In [128]:
lr_bp.mse_model

936.35000000000002

At this point I want to check against Minitab:

    Source         DF   Seq SS  Contribution   Adj SS   Adj MS  F-Value  P-Value
    Regression      2  1872.70        95.21%  1872.70   936.35   129.08    0.000
    Error          13    94.30         4.79%    94.30     7.25
    
Minitab is reporting the same values for $MSE$ and $MSR$. Now, let's calculate $F^*$:

In [129]:
fstar_bp = lr_bp.mse_model / lr_bp.mse_resid
fstar_bp

129.08324496288441

Turns out I can get this F-value and its associated p-value directly from `statsmodels`:

In [130]:
lr_bp.fvalue

129.08324496288441

In [131]:
lr_bp.f_pvalue

2.6582612670820174e-09

Just to be thorough, let's find $F_{crit} = F(1-\alpha; p-1; n-p)$:

In [132]:
fcrit_bp = st.f.ppf(0.99,2,lr_bp.nobs)
fcrit_bp

6.2262352803113856

Since 129.1 > 6.2, we conclude $H_a$: $\beta_1$ and $\beta_2$ do not both equal zero. In other words, at least one of the two variables (moisture and sweetness) are useful in regression.

### b. 

p-value = 2.66e-09

### c. 

First, let's see what `statsmodels` provides:

In [133]:
lr_bp.conf_int(alpha=0.01)

,0,1
const,28.624911,46.675089
moisture,3.517944,5.332056
sweetness,2.346762,6.403238


I do not know if `statsmodels` uses the Bonferroni method, so I will calculate the intervals manually for comparison. In order to use (6.52), I'll need to calculate B:

In [134]:
B_bp = st.t.ppf(1-.01/(2*2),lr_bp.nobs-3)
B_bp

3.3724679378582669

Get the coeffecients and standard errors:

In [135]:
# coefficients
lr_bp.params

const        37.650
moisture      4.425
sweetness     4.375
dtype: float64

In [136]:
# standard errors
lr_bp.bse

const        2.996103
moisture     0.301120
sweetness    0.673324
dtype: float64

Now, calculate the intervals:

In [137]:
# b1
b1_interv = (lr_bp.params['moisture'] - B_bp*lr_bp.bse['moisture'],
             lr_bp.params['moisture'] + B_bp*lr_bp.bse['moisture'])
b1_interv

(3.4094834484009602, 5.4405165515990408)

In [138]:
# b2
b2_interv = (lr_bp.params['sweetness'] - B_bp*lr_bp.bse['sweetness'],
             lr_bp.params['sweetness'] + B_bp*lr_bp.bse['sweetness'])
b2_interv

(2.1042359583483567, 6.6457640416516153)

Since these intervals are wider than the intervals given by `lr_bp.conf_int(alpha=0.01)`, that means `lr_bp.conf_int` is not using the Bonferroni method. Now I want to know what Minitab does:

    Coefficients

    Term       Coef  SE Coef      99% CI      T-Value  P-Value   VIF
    Constant  37.65     3.00  (28.62, 46.68)    12.57    0.000
    X1        4.425    0.301  (3.518, 5.332)    14.70    0.000  1.00
    X2        4.375    0.673  (2.347, 6.403)     6.50    0.000  1.00

It appears that Minitab is also _not_ using Bonferroni. So, just to be clear, the Bonferroni confidence intervals are 3.4094834484009602, 5.4405165515990408) for $\beta_1$ and (2.1042359583483567, 6.6457640416516153) for $\beta_2$. 

Interpretation: if I collect many samples of 16 data points and compute joint confidence intervals for $\beta_1$ and $\beta_2$ using a 99 percent family confidence coefficient, 99% of the joint intervals would contain the true values of $\beta_1$ and $\beta_2$ simultaneously. 

## 6.7

### a. 

In [139]:
lr_bp.rsquared

0.95205897305541431

This measures the proportionate reduction of total variation in $Y$ associated with the use of the two predictor variables, moisture and sweetness. 

### b.

According to Comment 2 in Kutner, p. 227, the coefficient of multiple determination will be equal to the coefficient of simple determination between $Y_i$ and $\hat Y_i$, so it is also 0.9521. Confirming with Minitab:

    Regression Analysis: Y versus FITS1 

    Model Summary

          S    R-sq  R-sq(adj)    PRESS  R-sq(pred)
    2.59533  95.21%     94.86%  127.759      93.50%

Yes, Minitab agrees. 

## 6.8

### a.

Using the predict function in Minitab:

    Prediction for Y 

    Variable  Setting
    X1              5
    X2              4

       Fit   SE Fit        99% CI              99% PI
    77.275  1.12669  (73.8811, 80.6689)  (68.4808, 86.0692)

So, $E\{Y_h\} = (73.8811, 80.6689)$ with 99% confidence. If we were to take many samples with a moisture value of 5 and a sweetness value of 4 and constructed a 99% confidence interval for brand liking for each sample, 99% of the intervals would contain the true value of brand liking for moisture of 5 and sweetness of 4. 

### b. 

Referrring to the Minitab output above, the 99% confidence prediction interval for brand liking when moisture is 5 and sweetness is 4 is (68.4808, 86.0692). We expect a new observation for moisture = 5 and sweetness = 4 to have a brand liking value within this range, with 99% confidence. 

## 22.

a. Yes: 

Let us define:

$$
\begin{aligned}
Z_{i1} &= X_{i1} \\
Z_{i2} &= log_{10}(X_{i2}) \\
Z_{i3} &= X_{i1}^2
\end{aligned}
$$

We can then write the equation as:

$$ Y_i = \beta_0 + \beta_1Z_{i1} + \beta_2Z_{i2} + \beta_3Z_{i3} + \varepsilon_i$$

b. No; No

c. Yes: 

$$
\begin{aligned}
Y_i &= log(\beta_1X_{i1}) + \beta_2X_{i2}+\varepsilon_i \\
Y_i &= log(\beta_1) + log(X_{i1}) + \beta_2X_{i2}+\varepsilon_i \\
\end{aligned}
$$

Let us define: 

$$
\begin{aligned}
\beta_0 &= log(\beta_1) \\
\beta_1 &= 1 \\
Z_{i1} &= log(X_{i1}) \\
Z_{i2} &= X_{i2} \\
\end{aligned}
$$

We can then write the equation as:

$$ Y_i = \beta_0 + \beta_1Z_{i1} + \beta_2Z_{i2} + \varepsilon_i$$

d. No; No

e. No; Yes: $Y' = ln(\frac{1}{Y}-1)$

## 6.23

### a.

Least squares criterion:

$$ Q = \sum_{i=1}^n(Y_i - \beta_1X_{i1} - \beta_2X_{i2})^2 \tag{1}$$

The least squares estimators are those values of $\beta_1$ and $\beta_2$ that minimize $Q$. To find them, we'll differentiate (1) with respect to $\beta_1$ and $\beta_2$:

$$
\begin{aligned}
\frac{\partial Q}{\partial\beta_1} &= -2\sum_{i=1}^nX_{i1}(Y_i - \beta_1X_{i1} - \beta_2X_{i2}) \\
\frac{\partial Q}{\partial\beta_2} &= -2\sum_{i=1}^nX_{i2}(Y_i - \beta_1X_{i1} - \beta_2X_{i2}) \\
\end{aligned}
$$

We then set these partial derivates equal to zero, using $b_1$ and $b_2$ to denote the least squares estimators of $\beta_1$ and $\beta_2$ (the values of $\beta_1$ and $\beta_2$ that minimize $Q$):

$$
\begin{aligned}
-2\sum_{i=1}^nX_{i1}(Y_i - b_1X_{i1} - b_2X_{i2}) &= 0 \\
-2\sum_{i=1}^nX_{i2}(Y_i - b_1X_{i1} - b_2X_{i2}) &= 0 \\
\end{aligned}
$$

Simplifying and expanding, we have:

$$
\begin{aligned}
\sum X_{i1}Y_i - b_1\sum X_{i1}^2 - b_2\sum X_{i1}X_{i2} &= 0 \\
\sum X_{i2}Y_i - b_1\sum X_{i1}X_{i2} - b_2\sum X_{i2}^2 &= 0 \\
\end{aligned}
$$

We then solve these equations simultaneously for $b_1$ and $b_2$. Each attempt that I made to solve this system of equations failed. It appears that I might have more success if I used the matrix representation from Kutner et al. p. 241 ($\mathbf{b=(X'X)^{-1}X'Y}$). However, I have run out of time and Dr. Wang said that I have done enough.


### b. 

Using (6.26), the likelihood function for this model is:

$$ L(\beta,\sigma^2) = \frac{1}{(2\pi\sigma^2)^{n/2}}exp\left[-\frac{1}{2\sigma^2}\sum_{i=1}^n(Y_i - \beta_1X_{i1} - \beta_2X_{i2})^2\right] $$

According to Kutner et al. (5th ed., p. 224), maximizing this likelihood function with respect to $\beta_1$ and $\beta_2$ leads to maximum likelihood estimators for $\beta_1$ and $\beta_2$ that equal the least squares estimators given above. To show this I would follow the same procedure as I did for the least squares estimators above: take partial derivatives of $L$ (actually, of $ln(L)$, as described in Kutner et al. p. 32) with respect to $\beta_1$ and $\beta_2$, equating the partials to zero and solving the resulting system of equations for $\hat \beta_1$ and $\hat \beta_2$. 